![](../images/TQ42_Banner.png)


# Welcome to TQ42!


## Begin by creating a client and selecting org, proj and exp

In [ ]:
from tq42.client import TQ42Client
from tq42.experiment_run import ExperimentRun, HardwareProto
from tq42.algorithm import AlgorithmProto

import json
from collections import Counter

from tq42_notebook_utils import Selector

In [ ]:
selector = Selector()

In [ ]:
with TQ42Client() as client: 
    selector.select_organization(client)

In [ ]:
with TQ42Client() as client: 
    selector.select_project(client)

In [ ]:
with TQ42Client() as client: 
    selector.select_experiment(client)

## Run a QuEnc experiment

This example uses the QuEnc experiment type to perform an optimization. Here the objective is to solve the max cut problem written as a QUBO. The outcome of the QuEnc experiment will be a circuit which can be sample to give an approximate solution to the QUBO.

The full explanation of the algorithm can be found in [*NISQ-compatible approximate quantum algorithm for unconstrained and constrained discrete optimization*](https://quantum-journal.org/papers/q-2023-11-21-1186/).

In [ ]:
print(f"Running experiment within: Org {selector.organization.id}, Proj {selector.project.id} and Exp {selector.experiment.id}`")

In [ ]:
parameters = {
    'parameters': {
        'qubo': [0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],  
        'number_layers': 5,
        'steps': 25,
        'velocity': 0.05,
        'optimizer': 'ADAM'
    },
    'inputs': {}
}

with TQ42Client() as client:
    quenc_run = ExperimentRun.create(
        client=client,
        algorithm=AlgorithmProto.TETRA_QUENC, 
        experiment_id=selector.experiment.id,
        compute=HardwareProto.SMALL, 
        parameters=parameters
    )

print(quenc_run.data)

## Poll the experiment run, and return results when finished

In [ ]:
quenc_result = quenc_run.poll()
print(quenc_result)

## Sample from the resulting circuit using a simulator

In [ ]:
circuit_storage_id = quenc_result.data.result.tetra_qu_enc_outcome.outputs.circuit.storage_id

In [ ]:
parameters = {
    'parameters': {
        'shots': 500,
        'backend': 'CIRQ_SIMULATOR'
    },
    'inputs': {
        'circuit': {'storage_id': circuit_storage_id}
    }
}

with TQ42Client() as client:
    run = ExperimentRun.create(
        client=client,
        algorithm=AlgorithmProto.CIRCUIT_RUN,
        experiment_id=selector.experiment.id,
        compute=HardwareProto.SMALL,
        parameters=parameters
    )

In [ ]:
circuit_run_result = run.poll()
print(circuit_run_result)

## Get the samples from the result and convert into an estimated probability distribution

In [ ]:
circuit_run_samples = json.loads(circuit_run_result.data.result.circuit_run_outcome.result)
circuit_run_samples

A single example will be a dictionary whose keys are qubit names and whose values are 1s and 0s. For example

```
{'qq(0)': 1, 'qq(1)': 0, 'qq(3)': 0, 'qq(2)': 1},
```

in which `qq(n)` is the $n$th qubit.

These samples are binary digit representations of integers. Thus, each sample must be convered to an integer. In this example the represented integer is

$$1 \cdot 2 ^ 0 + 0 \cdot 2 ^ 1 + 1 \cdot 2 ^ 2  + 0 \cdot 2 ^ 3 = 1 + 4 = 5$$

The following function converts a single sample into its represented integer:


In [ ]:
def sample_to_integer(sample):
    n = 0
    for qq_key, bit in sample.items(): 
        if bit:
            exponent = int(qq_key.replace("qq(", "").replace(")", ""))
            n += 1 << exponent
    return n

Each integer represents a possible measured state of the system. In this case, because there are four qubits in the system, there are $2^4 = 16$ possible states. We must count number of samples for each of these possible measured states and from the count estimate a probability distribution. 

In [ ]:
counter = Counter([sample_to_integer(sample) for sample in circuit_run_samples])

In [ ]:
state_size = 2 ** 4

In [ ]:
distribution = [0.0] * state_size

for n, count in counter.items():
    distribution[n] = count / len(circuit_run_samples)

distribution

Verify that this is actually a probablity distribution.

In [ ]:
sum(distribution)

## Interpret distribution as solution to QUBO

The first qubit is an ancilary qubit. If the ancilary qubit is 1 in a sample, then it means that sample is evidence that the choice represented by the remaining three qubits should be considered more likely. Because the ancilary qubit is qubit 0, those samples where the ancilary qubit is measured as 1 are the odd integers and those with the ancilary qubit measured 0 are the even integers. Hence, counting votes amounts to comparing the number of even and odd samples. This is how we interpret the circuit run results as a solution to the QUBO.

In [ ]:
solution = [distribution[2*i] < distribution[2*i+1] for i in range(state_size // 2)]
solution